In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,194 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,420 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,466 kB]
Hit:13 https://ppa.launchpadcontent.n

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
average_price_query = """
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
"""

# Execute the query
result = spark.sql(average_price_query)

# Show the result
result.show()



+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

average_price_query = """
    SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
"""

# Execute the query
result = spark.sql(average_price_query)

# Show the result
result.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
average_price_query = """
    SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
"""

# Execute the query
result = spark.sql(average_price_query)

# Show the result
result.show()



+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
average_price_query = """
    SELECT ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
"""

# Execute the query
result = spark.sql(average_price_query)

# Show the result
result.show()

# End time measurement
end_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.5926249027252197 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

average_price_query = """
    SELECT ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
"""

# Uncached runtime
uncached_start_time = time.time()

# Execute the query without using the cached data
result_uncached = spark.sql(average_price_query)

# Show the result
result_uncached.show()

# Uncached end time
uncached_end_time = time.time()

# Print the uncached runtime
print("Uncached Runtime: --- %s seconds ---" % (uncached_end_time - uncached_start_time))

# Spark SQL query to filter out view ratings with average price greater than or equal to $350,000
filtered_view_query = """
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price < 350000
    GROUP BY view
"""

# Cached runtime
cached_start_time = time.time()

# Execute the query using the cached data
result_cached = spark.sql(filtered_view_query)

# Show the result
result_cached.show()

# Cached end time
cached_end_time = time.time()

# Print the cached runtime
print("Cached Runtime: --- %s seconds ---" % (cached_end_time - cached_start_time))



+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

Uncached Runtime: --- 2.856229066848755 seconds ---
+----+-------------+
|view|average_price|
+----+-------------+
|  31|    238947.13|
|  34|    232111.68|
|  28|    233238.71|
|  26|    234223.99|
|  27|    235938.76|
|  44|    233404.96|
|  12|    233948.16|
|  22|    233355.73|
|  47|    240530.23|
|   1|    242193.79|
|  13|    229736.78|
|   6|    233480.09|
|  16|    235060.44|
|   3|    234470.83|
|  40|    238056.82|
|  20|     236196.6|
|  48|    234616.34|
|   5|    230727.41|
|  19|    233113.02|
|  41|    235149.62|
+----+-------------+
only showing top 20 rows

Cached Runtime: --- 0.8897151947021484 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Spark SQL query to filter out view ratings with an average price greater than or equal to $350,000
filtered_view_query = """
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price < 350000
    GROUP BY view
"""

# Execute the query using the cached data
result_cached = spark.sql(filtered_view_query)

# Show the result
result_cached.show()

# write out the data in parquet format
df.write.parquet('date_built',mode='overwrite')


+----+-------------+
|view|average_price|
+----+-------------+
|  31|    238947.13|
|  34|    232111.68|
|  28|    233238.71|
|  26|    234223.99|
|  27|    235938.76|
|  44|    233404.96|
|  12|    233948.16|
|  22|    233355.73|
|  47|    240530.23|
|   1|    242193.79|
|  13|    229736.78|
|   6|    233480.09|
|  16|    235060.44|
|   3|    234470.83|
|  40|    238056.82|
|  20|     236196.6|
|  48|    234616.34|
|   5|    230727.41|
|  19|    233113.02|
|  41|    235149.62|
+----+-------------+
only showing top 20 rows



In [15]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('date_built')

In [16]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView("parquet_home_sales")

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# Start time measurement for uncached version
uncached_start_time = time.time()

# Spark SQL query to filter out view ratings with an average price greater than or equal to $350,000
uncached_query = """
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM parquet_home_sales
    WHERE price < 350000
    GROUP BY view
"""

# Execute the uncached query
result_uncached = spark.sql(uncached_query)

# Show the result
result_uncached.show()

# End time measurement for uncached version
uncached_end_time = time.time()

# Print the runtime for uncached version
print("Uncached Runtime: --- %s seconds ---" % (uncached_end_time - uncached_start_time))

# Cache the Parquet DataFrame
p_df.cache()

# Start time measurement for cached version
cached_start_time = time.time()

# Spark SQL query to filter out view ratings with an average price greater than or equal to $350,000
cached_query = """
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM parquet_home_sales
    WHERE price < 350000
    GROUP BY view
"""

# Execute the cached query
result_cached = spark.sql(cached_query)

# Show the result
result_cached.show()

# End time measurement for cached version
cached_end_time = time.time()

# Print the runtime for cached version
print("Cached Runtime: --- %s seconds ---" % (cached_end_time - cached_start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  31|    238947.13|
|  34|    232111.68|
|  28|    233238.71|
|  26|    234223.99|
|  27|    235938.76|
|  44|    233404.96|
|  12|    233948.16|
|  22|    233355.73|
|  47|    240530.23|
|   1|    242193.79|
|  13|    229736.78|
|   6|    233480.09|
|  16|    235060.44|
|   3|    234470.83|
|  40|    238056.82|
|  20|     236196.6|
|  48|    234616.34|
|   5|    230727.41|
|  19|    233113.02|
|  41|    235149.62|
+----+-------------+
only showing top 20 rows

Uncached Runtime: --- 0.8350889682769775 seconds ---
+----+-------------+
|view|average_price|
+----+-------------+
|  31|    238947.13|
|  34|    232111.68|
|  28|    233238.71|
|  26|    234223.99|
|  27|    235938.76|
|  44|    233404.96|
|  12|    233948.16|
|  22|    233355.73|
|  47|    240530.23|
|   1|    242193.79|
|  13|    229736.78|
|   6|    233480.09|
|  16|    235060.44|
|   3|    234470.83|
|  40|    238056.82|
|  20|     236196.6|
|  48|    234616.3

In [22]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [23]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False